# Cервис для чтения книг по подписке

## Исследование таблиц

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Таблица - books

Содержит данные о книгах:
* book_id — идентификатор книги;
* author_id — идентификатор автора;
* title — название книги;
* num_pages — количество страниц;
* publication_date — дата публикации книги;
* publisher_id — идентификатор издателя.

In [3]:
query = '''
            SELECT *
            FROM books
        '''
book = pd.io.sql.read_sql(query, con = engine) 
book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


### Таблица - authors

Содержит данные об авторах:
* author_id — идентификатор автора;
* author — имя автор

In [4]:
query = '''
            SELECT *
            FROM authors
        '''
authors = pd.io.sql.read_sql(query, con = engine) 
authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
author_id    636 non-null int64
author       636 non-null object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


### Таблица - publishers

Содержит данные об издательствах:
* publisher_id — идентификатор издательства;
* publisher — название издательства;

In [5]:
query = '''
            SELECT *
            FROM publishers
        '''
publishers = pd.io.sql.read_sql(query, con = engine) 
publishers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
publisher_id    340 non-null int64
publisher       340 non-null object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


### Таблица - ratings

Содержит данные о пользовательских оценках книг:
* rating_id — идентификатор оценки;
* book_id — идентификатор книги;
* username — имя пользователя, оставившего оценку;
* rating — оценка книги.

In [6]:
query = '''
            SELECT *
            FROM ratings
        '''
ratings = pd.io.sql.read_sql(query, con = engine) 
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
rating_id    6456 non-null int64
book_id      6456 non-null int64
username     6456 non-null object
rating       6456 non-null int64
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


### Таблица - reviews

Содержит данные о пользовательских обзорах:
* review_id — идентификатор обзора;
* book_id — идентификатор книги;
* username — имя автора обзора;
* text — текст обзора.

In [7]:
query = '''
            SELECT *
            FROM reviews
        '''
reviews = pd.io.sql.read_sql(query, con = engine) 
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
review_id    2793 non-null int64
book_id      2793 non-null int64
username     2793 non-null object
text         2793 non-null object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


Выводы по данным в таблицах:
1. Пропущенных значений нет;
2. Типы данных не вызывают сомнения, за исключением даты публикации книги, но на данном этапе не будем менять тип данных, по необходимости изменим в дальнейшем.

## Cколько книг вышло после 1 января 2000 года?

In [8]:
book.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [9]:
query = ''' 
            SELECT count(book_id) AS Количество_книг
            FROM books
            WHERE publication_date > '01-01-2000'
          '''
query_1 = pd.io.sql.read_sql(query, con = engine)
print(query_1)

   Количество_книг
0              819


После 1-го января 2000 года вышло 819 книг. При условии, что всего в выборке 1000 книг и временной период с 1952 года. Книг явно стало производится больше!

## Количество обзоров и средняя оценка книг

In [10]:
query = '''
            SELECT books.book_id,ratings.average_ratings, reviews.count_reviews, books.title 
            FROM books
            LEFT JOIN (SELECT book_id, AVG(rating) AS average_ratings FROM ratings GROUP BY book_id) AS ratings ON books.book_id = ratings.book_id
            LEFT JOIN (SELECT book_id, COUNT(review_id) AS count_reviews FROM reviews GROUP BY book_id) AS reviews ON books.book_id = reviews.book_id
        '''
query_1 = pd.io.sql.read_sql(query, con = engine)
query_1

,book_id,average_ratings,count_reviews,title
0,652,4.500000,2.0,The Body in the Library (Miss Marple #3)
1,273,4.500000,2.0,Galápagos
2,51,4.250000,5.0,A Tree Grows in Brooklyn
3,951,4.000000,2.0,Undaunted Courage: The Pioneering First Missio...
4,839,4.285714,4.0,The Prophet
...,...,...,...,...
995,64,4.230769,4.0,Alice in Wonderland
996,55,5.000000,2.0,A Woman of Substance (Emma Harte Saga #1)
997,148,3.428571,3.0,Christine
998,790,3.500000,2.0,The Magicians' Guild (Black Magician Trilogy #1)


## Издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [16]:
query = '''
            SELECT publishers.publisher, books.book_id
            FROM publishers
            RIGHT JOIN (
            SELECT publisher_id, COUNT(book_id) AS book_id 
            FROM books 
            WHERE num_pages > 50 
            GROUP BY publisher_id) AS books ON publishers.publisher_id = books.publisher_id
            ORDER BY books.book_id DESC
            LIMIT 1
            '''
query_1 = pd.io.sql.read_sql(query, con = engine)
query_1

,publisher,book_id
0,Penguin Books,42


In [12]:
query = '''
            SELECT COUNT(book_id) AS book_id 
            FROM books 
            '''
query_1 = pd.io.sql.read_sql(query, con = engine)
query_1

,book_id
0,1000


Наибольшее число книг (не брошюры) было выпущено издательством Penguin Books. Это издательство выпустил чуть более 4% от всех представленных книг.

## Автор с самой высокой средней оценкой книг (учитываются только книги с 50 и более оценками)

In [13]:
query = '''
            SELECT author, AVG(rating) AS avg_rating
            FROM books
            RIGHT JOIN ratings ON books.book_id = ratings.book_id
            LEFT JOIN authors ON books.author_id = authors.author_id
            GROUP BY author
            HAVING COUNT(rating) > 50
            ORDER BY avg_rating DESC
            LIMIT 1
            '''
query_1 = pd.io.sql.read_sql(query, con = engine)
query_1

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.288462


Автор с самой высокой средней оценкой - J.K. Rowling/Mary GrandPré. 

## Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [14]:
query = '''
            SELECT AVG(count_user)
            FROM 
            (
            SELECT username,COUNT(review_id) AS count_user
            FROM reviews
            WHERE username IN(
            SELECT username--, COUNT(rating)
            FROM ratings
            GROUP BY username
            HAVING COUNT(rating) > 50
            )
            GROUP BY username) AS review_rating 

            '''
query_1 = pd.io.sql.read_sql(query, con = engine)
query_1

,avg
0,24.333333


Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок равно 24.3

Вывод:
1. Несомненно публиковтаь книги автора с самой высокой оценкой - J.K. Rowling/Mary GrandPré.;
2. Стоит обратить внимание на сотрудничество с издательством - Penguin Books - в среднем на 100 книг 4 выпускаются именно этим издательством;
3. Видим, что люди практически в два раза чаще только оценивают книги, но не пишут обзоры, поэтому необходимо простимулировать клиентов на обзоры(возможно давать баллы за обзоры, которые потом превращаются в денежные средства на покупку книг, однако обзоры должны проходить проверку, чтобы снизить шансы мошенничества).